In [3]:
!git clone https://github.com/k223kim/CryptocurrencyPredictionRNN.git

Cloning into 'CryptocurrencyPredictionRNN'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10 (delta 0), reused 10 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.


In [4]:
pwd

'/content'

In [0]:
import pandas as pd
import os

In [0]:
# df = pd.read_csv("/content/CryptocurrencyPredictionRNN/crypto_data/LTC-USD.csv", names=["time", "low", "high", "open", "close", "volume"])

In [7]:
# print(df.head())

         time        low       high       open      close      volume
0  1528968660  96.580002  96.589996  96.589996  96.580002    9.647200
1  1528968720  96.449997  96.669998  96.589996  96.660004  314.387024
2  1528968780  96.470001  96.570000  96.570000  96.570000   77.129799
3  1528968840  96.449997  96.570000  96.570000  96.500000    7.216067
4  1528968900  96.279999  96.540001  96.500000  96.389999  524.539978


In [0]:
SEQ_LEN = 60 #update once a minute
FUTURE_PERIOD_PREDICT = 3 #how many period forward? 1period = 1 min
RATIO_TO_PREDICT = "LTC-USD" #what are we predicting?

In [0]:
#create the target (since it is a supervised learning)

def classify(current, future):
  if float(future) > float(current):
    return 1 #you should buy this!
  else:
    return 0

In [43]:
main_df = pd.DataFrame()

ratios = ["BTC-USD", "LTC-USD", "ETH-USD", "BCH-USD"]
for ratio in ratios:
  dataset = f"/content/CryptocurrencyPredictionRNN/crypto_data/{ratio}.csv"
  
  df = pd.read_csv(dataset, names=["time", "low", "high", "open", "close", "volume"])
  print(df.head())
  
  #merge all dataframes
  #for all dataframes their columns are named the same
  #so we need a new column which distinguishes between those dataframes
  #inplace means that we don't have to redefine the dataframes
  df.rename(columns={"close": f"{ratio}_close", "volume":f"{ratio}_volume"}, inplace = True)

  df.set_index("time", inplace=True)
  df = df[[f"{ratio}_close", f"{ratio}_volume"]]

  if len(main_df) == 0: # if it is empty
    main_df = df
  else:
    main_df = main_df.join(df)
    
#get future price
#future price is based on the close column
#shift the close column up by how many periods forward you want to predict


# print(main_df.head())
# for c in main_df.columns:
#   print(c) #double check the columns


main_df["future"] = main_df[f"{RATIO_TO_PREDICT}_close"].shift(-FUTURE_PERIOD_PREDICT)

print(main_df[[f"{RATIO_TO_PREDICT}_close", "future"]].head())
#e.g. current price: 96.58 predicted future price: 96.5


         time          low         high         open        close    volume
0  1528968660  6489.549805  6489.560059  6489.560059  6489.549805  0.587100
1  1528968720  6487.370117  6489.560059  6489.549805  6487.379883  7.706374
2  1528968780  6479.410156  6487.370117  6487.370117  6479.410156  3.088252
3  1528968840  6479.410156  6479.419922  6479.419922  6479.410156  1.404100
4  1528968900  6475.930176  6479.979980  6479.410156  6479.979980  0.753000
         time        low       high       open      close      volume
0  1528968660  96.580002  96.589996  96.589996  96.580002    9.647200
1  1528968720  96.449997  96.669998  96.589996  96.660004  314.387024
2  1528968780  96.470001  96.570000  96.570000  96.570000   77.129799
3  1528968840  96.449997  96.570000  96.570000  96.500000    7.216067
4  1528968900  96.279999  96.540001  96.500000  96.389999  524.539978
         time        low   high        open      close     volume
0  1528968720  485.98999  486.5  486.019989  486.01001  26

In [49]:
#map the function to the new column "target"
#converts the output to a list which we will assign as a column
main_df["target"]= list(map(classify, main_df[f"{RATIO_TO_PREDICT}_close"], main_df["future"]))

print(main_df[[f"{RATIO_TO_PREDICT}_close", "future", "target"]].head(10))
#confirm if the target works (0-bad, 1-good)

            LTC-USD_close     future  target
time                                        
1528968660      96.580002  96.500000       0
1528968720      96.660004  96.389999       0
1528968780      96.570000  96.519997       0
1528968840      96.500000  96.440002       0
1528968900      96.389999  96.470001       1
1528968960      96.519997  96.400002       0
1528969020      96.440002  96.400002       0
1528969080      96.470001  96.400002       0
1528969140      96.400002  96.400002       0
1528969200      96.400002  96.400002       0
